<a href="https://colab.research.google.com/github/lqh52/TransferMarket/blob/main/DataCrawler_TransferMarket.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium

Reading package lists... Done
Building dependency tree       
Reading state information... Done
chromium-chromedriver is already the newest version (87.0.4280.66-0ubuntu0.18.04.1).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.
cp: '/usr/lib/chromium-browser/chromedriver' and '/usr/bin/chromedriver' are the same file


In [ ]:
from selenium import webdriver
import pandas as pd
from bs4 import BeautifulSoup
import re

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('-headless')
options.add_argument('-no-sandbox')
options.add_argument('disable-dev-shm-usage')

In [ ]:
player_id = []
player_name = []
market_value = []
player_age = []
player_team = []
goals = []
assists = []
appearances = []
points_per_game = []
for page in range(1,5):
  url = "https://www.transfermarkt.com/spieler-statistik/wertvollstespieler/marktwertetop?ajax=yw1&altersklasse=alle&ausrichtung=Sturm&jahrgang=0&kontinent_id=0&land_id=0&page={}&plus=1&spielerposition_id=alle".format(page)
  wd = webdriver.Chrome('chromedriver', options=options)
  wd.get(url)
  soup = BeautifulSoup(wd.page_source)
  table = soup.find_all('table',{'class': 'items'})
  rows = table[0].find_all('tr', {'class': ['odd', 'even']})

  tmp_player_id = []
  for row in rows:
    tds = row.find_all('td')
    tmp_player_id.append(tds[1].find('a', {'class': ['spielprofil_tooltip tooltipstered']}).get('id'))
    player_id.append(tds[1].find('a', {'class': ['spielprofil_tooltip tooltipstered']}).get('id'))
    player_name.append(tds[1].find('a', {'class': ['spielprofil_tooltip tooltipstered']}).text)
    market_value.append(tds[8].text.strip('\xa0'))
    player_age.append(tds[5].text)
    player_team.append(tds[7].find('img').get('alt'))

  for id in tmp_player_id:
    url_player = 'https://www.transfermarkt.com/kylian-mbappe/leistungsdatendetails/spieler/{}/saison/2020/verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1'.format(id)
    wd.get(url_player)
    goal = wd.find_element_by_xpath('/html/body/div[2]/div[11]/div/div[1]/div[4]/div/table/tfoot/tr/td[7]').text
    assist = wd.find_element_by_xpath('/html/body/div[2]/div[11]/div/div[1]/div[4]/div/table/tfoot/tr/td[8]').text
    appearance = wd.find_element_by_xpath('/html/body/div[2]/div[11]/div/div[1]/div[4]/div/table/tfoot/tr/td[5]').text
    ppg = wd.find_element_by_xpath('/html/body/div[2]/div[11]/div/div[1]/div[4]/div/table/tfoot/tr/td[6]').text
    goals.append(goal)
    assists.append(assist)
    appearances.append(appearance)
    points_per_game.append(ppg)

In [ ]:
df_2020 = pd.DataFrame({'id': player_id, 'name':player_name, 'age':player_age, 'team':player_team, 'appearances':appearances, 'points_per_game':points_per_game, 'goals':goals, 'assists':assists, 'market_value':market_value})

In [ ]:
df_2020

,id,name,age,team,appearances,points_per_game,goals,assists,market_value
0,342229,Kylian Mbappé,22,Paris Saint-Germain,33,2.30,26,9,€180.00m
1,68290,Neymar,29,Paris Saint-Germain,18,2.06,13,6,€128.00m
2,200512,Sadio Mané,28,Liverpool FC,36,1.72,12,5,€120.00m
3,148455,Mohamed Salah,28,Liverpool FC,39,1.59,25,4,€120.00m
4,132098,Harry Kane,27,Tottenham Hotspur,36,2.00,24,16,€120.00m
...,...,...,...,...,...,...,...,...,...
95,82009,Kevin Volland,28,AS Monaco,27,2.00,14,8,€28.00m
96,45320,Ángel Di María,33,Paris Saint-Germain,27,2.04,5,12,€28.00m
97,487465,Pedro Neto,21,Wolverhampton Wanderers,32,1.28,5,4,€27.00m
98,242631,Alex Iwobi,24,Everton FC,29,1.90,2,3,€25.50m


In [ ]:
df_2020.to_csv('2020', index = False)

In [ ]:
goals = []
assists = []
appearances = []
points_per_game = []
for id in player_id:
  url_player = 'https://www.transfermarkt.com/kylian-mbappe/leistungsdatendetails/spieler/{}/saison/2019/verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1'.format(id)
  wd.get(url_player)
  goal = wd.find_element_by_xpath('/html/body/div[2]/div[11]/div/div[1]/div[4]/div/table/tfoot/tr/td[7]').text
  assist = wd.find_element_by_xpath('/html/body/div[2]/div[11]/div/div[1]/div[4]/div/table/tfoot/tr/td[8]').text
  appearance = wd.find_element_by_xpath('/html/body/div[2]/div[11]/div/div[1]/div[4]/div/table/tfoot/tr/td[5]').text
  ppg = wd.find_element_by_xpath('/html/body/div[2]/div[11]/div/div[1]/div[4]/div/table/tfoot/tr/td[6]').text
  goals.append(goal)
  assists.append(assist)
  appearances.append(appearance)
  points_per_game.append(ppg)

In [ ]:
df_2019 = pd.DataFrame({'id': player_id, 'name':player_name, 'appearances':appearances, 'points_per_game':points_per_game, 'goals':goals, 'assists':assists})

In [ ]:
df_2019

,id,name,appearances,points_per_game,goals,assists
0,342229,Kylian Mbappé,37,2.57,30,18
1,68290,Neymar,27,2.52,19,12
2,200512,Sadio Mané,47,2.38,22,12
3,148455,Mohamed Salah,48,2.38,23,13
4,132098,Harry Kane,34,1.68,24,2
...,...,...,...,...,...,...
95,82009,Kevin Volland,40,1.70,12,10
96,45320,Ángel Di María,41,2.49,13,23
97,487465,Pedro Neto,44,1.73,5,5
98,242631,Alex Iwobi,29,1.24,2,1


In [ ]:
df_2019.to_csv('2019')

In [ ]:
team_rank = []
team_name = []
for page in range(1,5):
  url_team = "https://www.transfermarkt.com/spieler-statistik/wertvollstemannschaften/marktwertetop?ajax=yw1&page={}".format(page)
  wd.get(url_team)
  soup = BeautifulSoup(wd.page_source)
  table = soup.find_all('table',{'class': 'items'})
  rows = table[0].find_all('tr', {'class': ['odd', 'even']})

  for row in rows:
    tds = row.find_all('td')
    team_rank.append(tds[0].text)
    team_name.append(tds[2].find('a').text)

In [ ]:
df_team = pd.DataFrame({'rank':team_rank, 'team_name':team_name})

In [ ]:
df_team

,rank,team_name
0,1,Liverpool FC
1,2,Manchester City
2,3,Paris Saint-Germain
3,4,Bayern Munich
4,5,FC Barcelona
...,...,...
95,96,FC Augsburg
96,97,Celtic FC
97,98,FC Nantes
98,99,SV Werder Bremen


In [ ]:
df_team.to_csv('team')